In [4]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
from datetime import datetime
import os


In [9]:
ee.Authenticate()

TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000246049D6630>: Failed to resolve 'oauth2.googleapis.com' ([Errno 11001] getaddrinfo failed)"))

In [10]:
import ee
import geemap
import pandas as pd
import folium
import numpy as np
from datetime import datetime
from tqdm import tqdm

def initialize_gee():
    """Initialize Google Earth Engine"""
    try:
        ee.Initialize()
    except:
        ee.Authenticate()
        ee.Initialize()

# Define land cover classes globally
LANDCOVER_CLASSES = {
    10: ('Tree cover', '#006400'),
    20: ('Shrubland', '#ffbb22'),
    30: ('Grassland', '#ffff4c'),
    40: ('Cropland', '#f096ff'),
    50: ('Built-up', '#fa0000'),
    60: ('Bare/sparse vegetation', '#b4b4b4'),
    70: ('Snow and ice', '#f0f0f0'),
    80: ('Permanent water bodies', '#0064c8'),
    90: ('Herbaceous wetland', '#0096a0'),
    95: ('Mangroves', '#00cf75'),
    100: ('Moss and lichen', '#fae6a0')
}

def get_landcover_data():
    """
    Retrieve land cover data for Uttarakhand at 1km resolution
    """
    # Define Uttarakhand boundary
    uttarakhand = ee.FeatureCollection('FAO/GAUL/2015/level1') \
        .filter(ee.Filter.eq('ADM1_NAME', 'Uttarakhand'))

    # Get districts of Uttarakhand
    districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
        .filter(ee.Filter.eq('ADM1_NAME', 'Uttarakhand'))

    # Get ESA WorldCover data
    landcover = ee.ImageCollection("ESA/WorldCover/v100").first()

    # Clip to Uttarakhand boundary
    landcover_uttarakhand = landcover.clip(uttarakhand)

    def calculate_district_stats(image, districts):
        """Calculate land cover statistics for each district"""
        district_stats = []
        
        for district_feat in districts.getInfo()['features']:
            district_name = district_feat['properties']['ADM2_NAME']
            district_geom = ee.Feature(district_feat).geometry()
            
            district_stats_dict = {'District': district_name}
            
            for class_value, (class_name, _) in LANDCOVER_CLASSES.items():
                classification = image.eq(class_value)
                area = classification.multiply(ee.Image.pixelArea()) \
                    .reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=district_geom,
                        scale=1000,  # 1km resolution
                        maxPixels=1e9
                    ).get('Map').getInfo()
                
                # Convert to square kilometers
                area_sq_km = area / 1e6 if area is not None else 0
                district_stats_dict[class_name] = round(area_sq_km, 2)
            
            district_stats.append(district_stats_dict)
        
        return pd.DataFrame(district_stats)

    # Calculate district-wise statistics
    district_stats = calculate_district_stats(landcover_uttarakhand, districts)
    
    # Save district-wise statistics
    timestamp = datetime.now().strftime("%Y%m%d")
    district_stats.to_csv(f'uttarakhand_district_landcover_{timestamp}.csv', index=False)
    
    return landcover_uttarakhand, district_stats, districts

def export_pixel_data(landcover_img, region):
    """
    Export pixel-level data at 1km resolution
    """
    try:
        # Export the image to a TIF file using Earth Engine's export functionality
        export_params = {
            'image': landcover_img,
            'description': f'uttarakhand_landcover_1km_{datetime.now().strftime("%Y%m%d")}',
            'scale': 1000,  # 1km resolution
            'region': region.geometry(),
            'fileFormat': 'GeoTIFF',
            'maxPixels': 1e9
        }

        task = ee.batch.Export.image.toDrive(**export_params)
        task.start()
        
        print("Export task started. Please check your Google Drive for the output file.")
        print("Task ID:", task.id)
        
        return task.id

    except Exception as e:
        print(f"Error in export_pixel_data: {str(e)}")
        return None

def create_interactive_map(landcover_img, district_stats, districts):
    """
    Create an interactive map with land cover visualization
    """
    # Get the bounds of Uttarakhand
    uk_bounds = districts.geometry().bounds().getInfo()['coordinates'][0]
    center_lat = (uk_bounds[0][1] + uk_bounds[2][1]) / 2
    center_lon = (uk_bounds[0][0] + uk_bounds[2][0]) / 2

    # Create a folium map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

    # Add Earth Engine layer
    vis_params = {
        'bands': ['Map'],
        'min': 0,
        'max': 100,
        'palette': [color for _, color in LANDCOVER_CLASSES.values()]
    }

    # Add the land cover layer
    map_id_dict = ee.Image(landcover_img).getMapId(vis_params)
    folium.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name='Land Cover'
    ).add_to(m)

    # Add district boundaries and tooltips
    for idx, row in district_stats.iterrows():
        district_name = row['District']
        district_feature = next(
            (f for f in districts.getInfo()['features'] 
             if f['properties']['ADM2_NAME'] == district_name),
            None
        )
        
        if district_feature:
            district_geom = district_feature['geometry']
            
            # Create tooltip content
            tooltip_content = f"""
                <h4>{district_name}</h4>
                <table>
                    <tr><th>Land Cover Type</th><th>Area (km²)</th></tr>
                    {''.join(
                        f"<tr><td>{col}</td><td>{row[col]:.1f}</td></tr>"
                        for col in row.index if col != 'District'
                    )}
                </table>
            """
            
            # Add district polygon
            folium.GeoJson(
                district_geom,
                name=district_name,
                style_function=lambda x: {
                    'fillColor': 'none',
                    'color': 'black',
                    'weight': 1
                },
                tooltip=folium.Tooltip(tooltip_content)
            ).add_to(m)

    # Add a legend
    legend_html = """
        <div style="position: fixed; bottom: 50px; right: 50px; 
                    background-color: white; padding: 10px; border: 2px solid grey;
                    z-index: 1000;">
        <h4>Land Cover Types</h4>
    """
    for class_value, (class_name, color) in LANDCOVER_CLASSES.items():
        legend_html += f"""
            <div style="display: flex; align-items: center; margin-bottom: 5px;">
                <div style="width: 20px; height: 20px; background-color: {color};
                          margin-right: 5px;"></div>
                <div>{class_name}</div>
            </div>
        """
    legend_html += "</div>"
    m.get_root().html.add_child(folium.Element(legend_html))

    # Add layer control
    folium.LayerControl().add_to(m)

    # Save the map
    output_file = f'uttarakhand_landcover_map_{datetime.now().strftime("%Y%m%d")}.html'
    m.save(output_file)
    print(f"Map saved as {output_file}")

    return m

def process_geotiff_to_dataframe(geotiff_path):
    """
    Process the downloaded GeoTIFF file to create a structured DataFrame
    """
    import rasterio
    
    print("Processing GeoTIFF file...")
    
    with rasterio.open(geotiff_path) as src:
        # Read the raster data
        data = src.read(1)  # Read the first band
        
        # Get spatial information
        height, width = data.shape
        transform = src.transform
        
        # Create arrays for coordinates
        rows, cols = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
        
        # Convert pixel coordinates to geographic coordinates
        lons, lats = rasterio.transform.xy(transform, rows.ravel(), cols.ravel())
        
        # Create DataFrame
        df = pd.DataFrame({
            'longitude': lons,
            'latitude': lats,
            'land_cover_code': data.ravel()
        })
        
        # Add land cover class names
        df['land_cover_type'] = df['land_cover_code'].map({k: v[0] for k, v in LANDCOVER_CLASSES.items()})
        
        # Remove no-data values
        df = df.dropna()
        
        # Save to CSV
        output_file = f'uttarakhand_pixel_landcover_1km_{datetime.now().strftime("%Y%m%d")}.csv'
        df.to_csv(output_file, index=False)
        
        print(f"Processed data saved to {output_file}")
        return df

def main():
    try:
        # Initialize Earth Engine
        initialize_gee()
        
        # Get land cover data
        print("Retrieving land cover data...")
        landcover_img, district_stats, districts = get_landcover_data()
        
        print("\nDistrict-wise Land Cover Statistics:")
        print(district_stats)
        
        # Export pixel-level data
        task_id = export_pixel_data(landcover_img, districts)
        
        # Create interactive map
        print("\nCreating interactive map...")
        map_uttarakhand = create_interactive_map(landcover_img, district_stats, districts)
        
        return map_uttarakhand, district_stats, task_id
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        return None, None, None

if __name__ == "__main__":
    map_obj, stats, task_id = main()

Retrieving land cover data...

District-wise Land Cover Statistics:
             District  Tree cover  Shrubland  Grassland  Cropland  Built-up  \
0           Naini Tal     3526.15      27.78      69.32    337.50     31.10   
1   Udham Singh Nagar      739.45      15.41       2.62   1914.95     77.02   
2         Pithoragarh     2822.05       0.00    1408.63     22.67      4.56   
3              Almora     2550.95       0.29     487.10     53.09      5.20   
4           Bageshwar     1629.06       0.00     318.18     12.10      0.86   
5             Chamoli     2921.24       0.00    1629.50      2.58      0.86   
6           Champawat     1546.95       0.00      92.51     24.44      0.00   
7        Rudra Prayag      711.97       0.00     236.45      0.86      0.00   
8           Dehra Dun     2095.14       6.57     458.70    323.78    120.12   
9       Pauri Garhwal     4650.77       6.06     593.59     66.20     19.86   
10           Haridwar      674.44       9.84       8.12   1485.